# 9장 트리 기반 회귀
---
    - CART 알고리즘
    - 회귀와 모델 트리
    - 트리 가지치기 알고리즘
    - 파이썬으로 GUI구축하기
    
    데이터가 비선형의 형태일때, 데이터를 세분화 하여 회귀 기술을 적용할 수 있다. 
     

## 9.1 지역적으로 복잡한 데이터 모델링하기
---
    - 트리 구축
     3장에서 사용했던 알고리즘은 ID3: 속성만 선택하여 분할, 연속적인 속성을 직접적으로 제어할 수 없음
     => 이진분할: 기준에 해당하는 데이터는 왼쪽, 아니면 오른쪽

    - 트리 기반 회귀
    장점: 비선형처럼 복잡한 데이터에 적합하다.
    단점: 결과를 해석하기가 어렵다.
    활용: 수치형 값, 명목형 값
    
    - 트리 기반 회귀의 일반적인 접근방법
    수집: 모든 방법
    준비:수치형 값이 요구된다. 명목형 값이라면 이를 이진형 값으로 변경하는 것이 좋다.
    분석: 데이터를 2차원 플롯으로 시각화하고 딕셔너리처럼 트리를 생성한다.
    훈련: 단말 노드에서 모델을 가지고 구축하는 데 가장 많은 시간을 보낸다.
    검사: 모델의 성능을 알아보기 위해 검사 데이터에 R2값을 사용할 것이다.
    사용: 예측을 위해 트리를 사용할 것이다. 이 결과를 가지고 거의 모든 것을 할 수 있다.


## 9.2 연속적이고 이산적인 속성으로 트리 구축하기
---
    Feature: 트리에서 분할에 사용될 속성을 나타내는 기호
    Value: 분할에 사용된 속성의 값
    Right: 오른쪽 하위 트리. 알고리즘이 또 다른 분할을 필요로 하지 않을 경우, 하나의 단일한 값이 될 수도 있다.
    Left: 왼쪽 하위 트리. 오른쪽 하위 트리와 유사함
        
    - 트리의 유형
    회귀 트리: 각 노드에 단일한 값
    모델 트리: 각 단말 도느에 선형 방정식
     


In [1]:
from numpy import *

class treeNode():
    def __init__(self, feat, val, right, left):
        featureToSplitOn = feat
        valueOfSplit = val
        rightBranch = right
        leftBranch = left

def loadDataSet(fileName):      #general function to parse tab -delimited floats
    dataMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float,curLine) #map all elements to float()
        dataMat.append(fltLine)
    return dataMat

def binSplitDataSet(dataSet, feature, value):
    mat0 = dataSet[nonzero(dataSet[:,feature] > value)[0],:]
    mat1 = dataSet[nonzero(dataSet[:,feature] <= value)[0],:]
    return mat0,mat1

testMat = mat(eye(4))
print testMat
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)
print mat0, mat1

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
[[ 0.  1.  0.  0.]] [[ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


## 9.3 회귀를 위해 CART 사용하기
---
    기존: 섀넌 엔트로피 사용
    
    -CART(Classification And Regression Trees) 알고리즘
    CART 알고리즘은 지니 지수(Gini Index) 또는 분산의 감소량을 사용하여 나무의 가지를 이진(Binary) 분리한다. (범주형 변수에 대해서는 지니 지수를 사용하고, 연속형 변수에 대해서는 분산의 감소량을 사용한다.) 참고자료 : 사례로 배우는 데이터마이닝 [자유아카데미, 최종후/소선하] p.28
    이진 분할 구축, 연속적인 변화 제어
    무질서도 측정: 평균값 계산 -> 편차의 제곱

### 9.3.1 트리 구축하기
---

In [2]:
def regLeaf(dataSet):#returns the value used for each leaf
    return mean(dataSet[:,-1])

def regErr(dataSet):
    return var(dataSet[:,-1]) * shape(dataSet)[0]

def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    tolS = ops[0]; tolN = ops[1]
    #if all the target variables are the same value: quit and return value
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1: #exit cond 1
        return None, leafType(dataSet)
    m,n = shape(dataSet)
    #the choice of the best feature is driven by Reduction in RSS error from mean
    S = errType(dataSet)
    bestS = inf; bestIndex = 0; bestValue = 0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:,featIndex].A1):
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN): continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS: 
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    #if the decrease (S-bestS) is less than a threshold don't do the split
    if (S - bestS) < tolS: 
        return None, leafType(dataSet) #exit cond 2
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):  #exit cond 3
        return None, leafType(dataSet)
    return bestIndex,bestValue#returns the best feature to split on
                              #and the value used for that split

def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):#assume dataSet is NumPy Mat so we can array filtering
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)#choose the best split
    if feat == None: return val #if the splitting hit a stop condition return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet, rSet = binSplitDataSet(dataSet, feat, val)
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree  


### 9.3.2 코드 실행하기
---
![img](l.png)

In [3]:
myDat = loadDataSet('ex00.txt')
myMat = mat(myDat)
# set(myMat[:,1].A1)
# # {i for i in }
createTree(myMat)

{'left': 1.0180967672413792,
 'right': -0.044650285714285719,
 'spInd': 0,
 'spVal': 0.48813000000000001}

![img](2.png)

In [4]:
myDat = loadDataSet('ex0.txt')
myMat = mat(myDat)
# set(myMat[:,1].A1)
# # {i for i in }
createTree(myMat)

{'left': {'left': {'left': 3.9871631999999999,
   'right': 2.9836209534883724,
   'spInd': 1,
   'spVal': 0.79758300000000004},
  'right': 1.980035071428571,
  'spInd': 1,
  'spVal': 0.58200200000000002},
 'right': {'left': 1.0289583666666666,
  'right': -0.023838155555555553,
  'spInd': 1,
  'spVal': 0.19783400000000001},
 'spInd': 1,
 'spVal': 0.39434999999999998}

## 9.4 트리 가지치기
---
    과적합을 방지하기 위한 가지치기
    사전 가지치기
    사후 가지치기

### 9.5.1 사전 가지치기
---
    -사전 가지치기
    ChooseBestSplit()에서 종료조건 설정

In [5]:
createTree(myMat, ops = (0,1))

{'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': 4.2372350000000001,
         'right': {'left': {'left': {'left': {'left': 3.8087529999999998,
             'right': 3.8113929999999998,
             'spInd': 1,
             'spVal': 0.99677499999999997},
            'right': 3.7377500000000001,
            'spInd': 1,
            'spVal': 0.99560800000000005},
           'right': {'left': {'left': {'left': {'left': {'left': 3.9936790000000002,
                'right': 4.0118340000000003,
                'spInd': 1,
                'spVal': 0.99180999999999997},
               'right': 4.0521370000000001,
               'spInd': 1,
               'spVal': 0.98676900000000001},
              'right': 3.892763,
              'spInd': 1,
              'spVal': 0.98315300000000005},
             'right': 3.7293759999999998,
             'spInd': 1,
             'spVal': 0.977989},
            'right': 4.1494090000000003,
            'spInd': 1,
            'sp

![img](3.png)

In [6]:
myDat2 = loadDataSet('ex2.txt')
myMat2 = mat(myDat2)
createTree(myMat2)

{'left': {'left': {'left': {'left': 105.24862350000001,
    'right': 112.42895575000001,
    'spInd': 0,
    'spVal': 0.95851200000000003},
   'right': {'left': {'left': {'left': {'left': 87.310387500000004,
       'right': {'left': {'left': 96.452866999999998,
         'right': {'left': 104.82540899999999,
          'right': {'left': 95.181792999999999,
           'right': 102.25234449999999,
           'spInd': 0,
           'spVal': 0.87288299999999996},
          'spInd': 0,
          'spVal': 0.89299899999999999},
         'spInd': 0,
         'spVal': 0.91097499999999998},
        'right': 95.275843166666661,
        'spInd': 0,
        'spVal': 0.85497000000000001},
       'spInd': 0,
       'spVal': 0.94422099999999998},
      'right': {'left': 81.110151999999999,
       'right': 88.784498800000009,
       'spInd': 0,
       'spVal': 0.81160200000000005},
      'spInd': 0,
      'spVal': 0.83302600000000004},
     'right': 102.35780185714285,
     'spInd': 0,
     'spVal': 0.79

In [7]:
createTree(myMat2, ops = (10000,4))

{'left': 101.35815937735848,
 'right': -2.6377193297872341,
 'spInd': 0,
 'spVal': 0.49917099999999998}

### 9.4.2 사후 가지치기
---
    사전 가지치기는 변수를 직접 조정해야함
    
    - 사후 가지치기
    검사 / 훈련 집합으로 분할
    모두 전개
    검사집합으로 오류를 줄이는 방향으로 병합

In [8]:
def isTree(obj):
    return (type(obj).__name__=='dict')

def getMean(tree):
    if isTree(tree['right']): tree['right'] = getMean(tree['right'])
    if isTree(tree['left']): tree['left'] = getMean(tree['left'])
    return (tree['left']+tree['right'])/2.0
    
def prune(tree, testData):
    if shape(testData)[0] == 0: return getMean(tree) #if we have no test data collapse the tree
    if (isTree(tree['right']) or isTree(tree['left'])):#if the branches are not trees try to prune them
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
    if isTree(tree['left']): tree['left'] = prune(tree['left'], lSet)
    if isTree(tree['right']): tree['right'] =  prune(tree['right'], rSet)
    #if they are now both leafs, see if we can merge them
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        errorNoMerge = sum(power(lSet[:,-1] - tree['left'],2)) +\
            sum(power(rSet[:,-1] - tree['right'],2))
        treeMean = (tree['left']+tree['right'])/2.0
        errorMerge = sum(power(testData[:,-1] - treeMean,2))
        if errorMerge < errorNoMerge: 
            print "merging"
            return treeMean
        else: return tree
    else: return tree

In [9]:
myTree = createTree(myMat2, ops=(0,1))
myDatTest = loadDataSet('ex2test.txt')
myMat2Test = mat(myDatTest)
prune(myTree, myMat2Test)

merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging


{'left': {'left': {'left': {'left': 92.523991499999994,
    'right': {'left': {'left': {'left': 112.386764,
       'right': 123.559747,
       'spInd': 0,
       'spVal': 0.96039799999999997},
      'right': 135.83701300000001,
      'spInd': 0,
      'spVal': 0.95851200000000003},
     'right': 111.2013225,
     'spInd': 0,
     'spVal': 0.956951},
    'spInd': 0,
    'spVal': 0.96596899999999997},
   'right': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': {'left': 96.41885225,
              'right': 69.318648999999994,
              'spInd': 0,
              'spVal': 0.94882200000000005},
             'right': {'left': {'left': 110.03503850000001,
               'right': {'left': 65.548417999999998,
                'right': {'left': 115.75399400000001,
                 'right': {'left': {'left': 94.396114499999996,
                   'right': 85.005351000000005,
                   'spInd': 0,
                   'spVal': 0.912161},
          

많이 줄어들지 않았다. 사후 가지치기가 사전 가지치기보다 효과적이지 않다는 것을 나타낸다(?)

## 9.5 모델 트리
---
    -모델 트리
    구간별 선형(Piecewise linear)를 처리하는 방법
![img](4.png)

In [10]:
def linearSolve(dataSet):   #helper function used in two places
    m,n = shape(dataSet)
    X = mat(ones((m,n))); Y = mat(ones((m,1)))#create a copy of data with 1 in 0th postion
    X[:,1:n] = dataSet[:,0:n-1]; Y = dataSet[:,-1]#and strip out Y
    xTx = X.T*X
    if linalg.det(xTx) == 0.0:
        raise NameError('This matrix is singular, cannot do inverse,\n\
        try increasing the second value of ops')
    ws = xTx.I * (X.T * Y)
    return ws,X,Y

def modelLeaf(dataSet):#create linear model and return coeficients
    ws,X,Y = linearSolve(dataSet)
    return ws

def modelErr(dataSet):
    ws,X,Y = linearSolve(dataSet)
    yHat = X * ws
    return sum(power(Y - yHat,2))

In [11]:
myMat2 = mat(loadDataSet('exp2.txt'))
createTree(myMat2, modelLeaf, modelErr, (1,10))

{'left': matrix([[  1.69855694e-03],
         [  1.19647739e+01]]), 'right': matrix([[ 3.46877936],
         [ 1.18521743]]), 'spInd': 0, 'spVal': 0.28547699999999998}

y = 1.69855694e-03 + 1.19647739e+01x  
y = 3.46877936 + 1.18521743x

y = 0 + 12x  
y = 3.5 + 1.0 x

## 9.6 예제: 일반 회귀와 트리 방법 비교
---
![img](5.png)

In [12]:
def regTreeEval(model, inDat):
    return float(model)

def modelTreeEval(model, inDat):
    n = shape(inDat)[1]
    X = mat(ones((1,n+1)))
    X[:,1:n+1]=inDat
    return float(X*model)

def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree): return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)
        
def createForeCast(tree, testData, modelEval=regTreeEval):
    m=len(testData)
    yHat = mat(zeros((m,1)))
    for i in range(m):
        yHat[i,0] = treeForeCast(tree, mat(testData[i]), modelEval)
    return yHat

In [13]:
trainMat = mat(loadDataSet('bikeSpeedVsIq_train.txt'))
testMat = mat(loadDataSet('bikeSpeedVsIq_test.txt'))
myTree = createTree(trainMat, ops = (1,20))
yHat = createForeCast(myTree, testMat[:,0])
corrcoef(yHat, testMat[:,1], rowvar = 0)[0, 1]

0.96408523182221406

In [14]:
myTree = createTree(trainMat, modelLeaf, modelErr, ops = (1,20))
yHat = createForeCast(myTree, testMat[:,0], modelTreeEval)
corrcoef(yHat, testMat[:,1], rowvar = 0)[0, 1]

0.97604121913805997

In [15]:
ws, X, Y = linearSolve(trainMat)
ws

matrix([[ 37.58916794],
        [  6.18978355]])

In [16]:
for i in range(shape(testMat)[0]):
    yHat[i] = testMat[i, 0] * ws[1,0] + ws[0,0]
corrcoef(yHat, testMat[:,1], rowvar = 0)[0, 1]

0.94346842356747629

## 9.7 파이썬에서 GUI를 생성하기 위해 Tkinter 사용하기
---

In [17]:
from Tkinter import *
root = Tk()
myLabel = Label(root, text="Hello World")
myLabel.grid()
root.mainloop()

In [18]:
from numpy import *
from Tkinter import *
import regTrees
def reDraw(tolS,tolN):
    pass
def drawNewTree():
    pass

root=Tk()
Label(root, text="Plot Place Holder").grid(row=0, columnspan=3)
Label(root, text="tolN").grid(row=1, column=0)
tolNentry = Entry(root)
tolNentry.grid(row=1, column=1)
tolNentry.insert(0,'10')
Label(root, text="tolS").grid(row=2, column=0)
tolSentry = Entry(root)
tolSentry.grid(row=2, column=1)
tolSentry.insert(0,'1.0')
Button(root, text="ReDraw", command=drawNewTree).grid(row=1, column=2,rowspan=3)
# Button(root, text='Quit',fg="black", command=root.quit).grid(row=1,column=2)
chkBtnVar = IntVar()
chkBtn = Checkbutton(root, text="Model Tree", variable = chkBtnVar)
chkBtn.grid(row=3, column=0, columnspan=2)
reDraw.rawDat = mat(regTrees.loadDataSet('sine.txt'))
reDraw.testDat = arange(min(reDraw.rawDat[:,0]),max(reDraw.rawDat[:,0]),0.01)
reDraw(1.0, 10)
root.mainloop()


In [19]:
root=Tk()

reDraw.f = Figure(figsize=(5,4), dpi=100)
reDraw.canvas = FigureCanvasTkAgg(reDraw.f, master=root)
reDraw.canvas.show()
reDraw.canvas.get_tk_widget().grid(row=0, columnspan=3)

Label(root, text="canvas").grid(row=0, columnspan=3)
Label(root, text="tolN").grid(row=1, column=0)
tolNentry = Entry(root)
tolNentry.grid(row=1, column=1)
tolNentry.insert(0,'10')
Label(root, text="tolS").grid(row=2, column=0)
tolSentry = Entry(root)
tolSentry.grid(row=2, column=1)
tolSentry.insert(0,'1.0')
Button(root, text="ReDraw", command=drawNewTree).grid(row=1, column=2,rowspan=3)
chkBtnVar = IntVar()
chkBtn = Checkbutton(root, text="Model Tree", variable = chkBtnVar)
chkBtn.grid(row=3, column=0, columnspan=2)
reDraw.rawDat = mat(regTrees.loadDataSet('sine.txt'))
reDraw.testDat = arange(min(reDraw.rawDat[:,0]),max(reDraw.rawDat[:,0]),0.01)
reDraw(1.0, 10)
root.mainloop()


NameError: name 'Figure' is not defined

In [20]:
from numpy import *

from Tkinter import *
import regTrees

import matplotlib
matplotlib.use('TkAgg') #백엔드 변경, raster images로 만드는 C++라이브러리
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

root = Tk()
rawDat = mat(loadDataSet('sine.txt'))
testDat = arange(min(rawDat[:,0]),max(rawDat[:,0]),0.01)

def reDraw(tolS,tolN):
    reDraw.f.clf()        # clear the figure
    reDraw.a = reDraw.f.add_subplot(111)
    if chkBtnVar.get():
        if tolN < 2: tolN = 2
        myTree = createTree(rawDat, modelLeaf, modelErr, (tolS,tolN))
        yHat = createForeCast(myTree, testDat, modelTreeEval)
    else:
        myTree=createTree(rawDat, ops=(tolS,tolN))
        yHat = createForeCast(myTree, testDat)
    reDraw.a.scatter(rawDat[:,0], rawDat[:,1], s=5) # use scatter for data set
    reDraw.a.plot(testDat, yHat, linewidth=2.0) # use plot for yHat
    reDraw.canvas.show()
    
def getInputs():
    try: tolN = int(tolNentry.get())
    except: 
        tolN = 10 
        print "enter Integer for tolN"
        tolNentry.delete(0, END)
        tolNentry.insert(0,'10')
    try: tolS = float(tolSentry.get())
    except: 
        tolS = 1.0 
        print "enter Float for tolS"
        tolSentry.delete(0, END)
        tolSentry.insert(0,'1.0')
    return tolN,tolS

def drawNewTree():
    tolN,tolS = getInputs()#get values from Entry boxes
    reDraw(tolS,tolN)
    
root=Tk()

reDraw.f = Figure(figsize=(5,4), dpi=100) #create canvas
reDraw.canvas = FigureCanvasTkAgg(reDraw.f, master=root)
reDraw.canvas.show()
reDraw.canvas.get_tk_widget().grid(row=0, columnspan=3)

Label(root, text="tolN").grid(row=1, column=0)
tolNentry = Entry(root)
tolNentry.grid(row=1, column=1)
tolNentry.insert(0,'10')
Label(root, text="tolS").grid(row=2, column=0)
tolSentry = Entry(root)
tolSentry.grid(row=2, column=1)
tolSentry.insert(0,'1.0')
Button(root, text="ReDraw", command=drawNewTree).grid(row=1, column=2, rowspan=3)
chkBtnVar = IntVar()
chkBtn = Checkbutton(root, text="Model Tree", variable = chkBtnVar)
chkBtn.grid(row=3, column=0, columnspan=2)
reDraw(1.0, 10)

root.mainloop()